In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from isic_helper import compute_pauc, compute_auc

In [2]:
model_names = ["lgb", "resnet18"]
versions = ["v1", "v1"]
target_column = "final_target"

In [3]:
subs = {}
cv_pauc_oof_avg = {}
for model_name, version in zip(model_names, versions):
    subs[model_name] = pd.read_csv(f"/kaggle/input/isic-scd-{model_name}-train/oof_preds_{model_name}_{version}.csv")
    cv_pauc_oof_avg[model_name] = {}
cv_pauc_oof_avg["ensemble"] = {}

# wts = [1/len(subs)]*len(subs)
wts = [0.65, 0.35]
folds = np.sort(subs[model_names[0]]["fold"].unique())
for fold in folds:
    fold_ens_preds = []
    for idx, (version, (model_name, df)) in enumerate(zip(versions, subs.items())):
        fold_df = df[df["fold"] == fold]
        fold_model_preds = fold_df[f"oof_{model_name}_{version}"]
        cv_pauc_oof_avg[model_name][f"fold_{fold}"] = compute_pauc(fold_df[target_column], fold_model_preds)
        fold_ens_preds.append(wts[idx] * fold_df[f"oof_{model_name}_{version}"].rank(pct=True).values)
    cv_pauc_oof_avg["ensemble"][f"fold_{fold}"] = compute_pauc(fold_df[target_column], np.sum(fold_ens_preds, axis=0))

In [4]:
for model_name in model_names:
    print(f"{model_name}:\nFold scores: {cv_pauc_oof_avg[model_name]}\n Avg score: {np.mean(list(cv_pauc_oof_avg[model_name].values()))}")
print(f"Ensemble:\nFold scores: {cv_pauc_oof_avg['ensemble']}\n Avg score: {np.mean(list(cv_pauc_oof_avg['ensemble'].values()))}")

lgb:
Fold scores: {'fold_1': 0.16411489480109465, 'fold_2': 0.16163455058930135, 'fold_3': 0.15529609327226662, 'fold_4': 0.1696645181016086, 'fold_5': 0.16317145182032303}
 Avg score: 0.16277630171691884
resnet18:
Fold scores: {'fold_1': 0.15586849727633634, 'fold_2': 0.15323922173948376, 'fold_3': 0.14612993098800708, 'fold_4': 0.15783404674844936, 'fold_5': 0.15046869195347348}
 Avg score: 0.15270807774115
Ensemble:
Fold scores: {'fold_1': 0.16939685593615736, 'fold_2': 0.16963936368121854, 'fold_3': 0.16170013789919002, 'fold_4': 0.17493245435599203, 'fold_5': 0.16782189353789065}
 Avg score: 0.16869814108208975
